# Iterables, Iterators, and Generators

## Introduction: Examples

In [1]:
values = [1, 2, 3]

for v in values:
    print(v)

1
2
3


In [2]:
string = "I hurt myself today to see if I still feel"

for letter in string:
    print(letter, end='')

I hurt myself today to see if I still feel

In [3]:
number = 124567

try:
    for digit in number:
        print(digit)
except TypeError as error:
    print(f'Failed to iterate over integer number :( "{error}"')

Failed to iterate over integer number :( "'int' object is not iterable"


## Going Deeper: Under the Hood

### Infinite Iterator

In [4]:
# Отдаёт значения на каждой итерации

class Iterator:
    def __init__(self, value):
        self.value = value
    
    def __next__(self):
        return self.value


# Умеет порождать того, кто умеет отдавать

class Iterable:
    def __init__(self, value):
        self.value = value
    
    def __iter__(self):
        return Iterator(value=self.value)

Итерирование с помощью цикла `for`:

In [5]:
max_num_times = 5
num_times = 0

iterable = Iterable(value="Hello world!")

for value in iterable:
    if num_times >= max_num_times:
        break

    print(value)
    
    num_times += 1

Hello world!
Hello world!
Hello world!
Hello world!
Hello world!


Равносильный вариант с помощью цикла `while`:

In [6]:
max_num_times = 5
num_times = 0

iterable = Iterable(value="Hello world!")
iterator = iter(iterable)

while True:
    value = next(iterator)
    
    if num_times >= max_num_times:
        break

    print(value)
    
    num_times += 1

Hello world!
Hello world!
Hello world!
Hello world!
Hello world!


In [7]:
del Iterable, Iterator

### Limited Iterator

In [8]:
class Iterator:
    def __init__(self, value, num_times: int):
        self.value = value
        self.num_times = num_times
        self._current_num_times = 0
    
    def __next__(self):
        if self._current_num_times >= self.num_times:
            raise StopIteration()
        
        self._current_num_times += 1

        return self.value


class Iterable:
    def __init__(self, value, num_times: int = 5):
        self.value = value
        self.num_times = num_times
    
    def __iter__(self):
        return Iterator(value=self.value, num_times=self.num_times)

Использование итератора в цикле `for`:

In [9]:
iterable = Iterable(value="Hello world!")

for value in iterable:
    print(value)

Hello world!
Hello world!
Hello world!
Hello world!
Hello world!


И снова — равносильный вариант с помощью цикла `while`:

In [10]:
iterable = Iterable(value="Hello world!")
iterator = iter(iterable)

while True:
    try:
        value = next(iterator)
    except StopIteration:
        break

    print(value)

Hello world!
Hello world!
Hello world!
Hello world!
Hello world!


In [11]:
del Iterable, Iterator

### Two in One for Simplicity

In [12]:
class IterableAndIterator:
    def __init__(self, value, num_times: int = 5):
        self.value = value
        self.num_times = num_times
        self._current_num_times = 0
    
    def __iter__(self):
        return self

    def __next__(self):
        if self._current_num_times >= self.num_times:
            raise StopIteration()
        
        self._current_num_times += 1

        return self.value

In [13]:
iterable = IterableAndIterator(value="Hello world!")

for value in iterable:
    print(value)

Hello world!
Hello world!
Hello world!
Hello world!
Hello world!


Из-за того, что `__iter__` возвращает `self`, второй раз по "отработанному" итератору пройтись уже не получится:

In [14]:
for value in iterable:
    print(value)

In [15]:
del IterableAndIterator

## Generators

Как функция, но не функция:

In [16]:
def repeate(value, num_times: int = 5):
    current_num_times = 0
    
    while True:
        if current_num_times >= num_times:
            break

        yield value  # Именно это делает генератор генератором!
        
        current_num_times += 1

In [17]:
generator = repeate(value="Hello world!")

for value in generator:
    print(value)

Hello world!
Hello world!
Hello world!
Hello world!
Hello world!


Снова повторное итерирование по тому же генератору ни к чему не приводит:

In [18]:
for value in generator:
    print(value)

Потому что `__iter__` генератора возвращает его же самого:

In [19]:
generator == iter(generator)

True

И снова — аналог через цикл `while`:

In [20]:
generator = repeate(value="Hello world!")

while True:
    try:
        value = next(generator)
    except StopIteration:
        break

    print(value)

Hello world!
Hello world!
Hello world!
Hello world!
Hello world!


Eщё одна версия генератора с промежуточными принтами, чтоб посмотреть, "что там происходит":

In [21]:
def repeate(value, num_times: int = 5):
    print('Started.')
    
    current_num_times = 0
    
    while True:
        if current_num_times >= num_times:
            break
        
        print('Before yield.')

        yield value  # Именно это делает генератор генератором!
        
        print('After yield.')
        
        current_num_times += 1
    
    print('Finished.')

In [22]:
generator = repeate(value="Hello world!")

In [23]:
next(generator)

Started.
Before yield.


'Hello world!'

In [24]:
next(generator)

After yield.
Before yield.


'Hello world!'

In [25]:
del repeate

## Examples of Popular Built-in Iterators

In [26]:
for i in range(3):
    print(i)

0
1
2


In [27]:
letters = ['K', 'I', 'S', 'S']
meanings = ['keep', 'it', 'simple', 'stupid']

for letter, meaning in zip(letters, meanings):
    print(f'{letter}: {meaning}')

K: keep
I: it
S: simple
S: stupid


In [28]:
contestant_names = ['Lera', 'Dasha', 'Nadya']
places_in_the_beauty_contest = [1, 2, 3]

for place, name in zip(places_in_the_beauty_contest, contestant_names):
    print(f'{name}: {place}')

Lera: 1
Dasha: 2
Nadya: 3


In [29]:
names = ['Lera', 'Dasha', 'Nadya']

for index, name in enumerate(names):
    print(f'{name}: {index}')

Lera: 0
Dasha: 1
Nadya: 2


### Filter

In [30]:
values = [1, -1, 2, -2, 3, -3]

nonzero_values = filter(lambda x: x >= 0, values)

In [31]:
nonzero_values

In [32]:
list(nonzero_values)

[1, 2, 3]

Равносильный (почти) вариант сделать то же самое без `filter` (и без лямбда функции):

In [33]:
nonzero_values = [v for v in values if v >= 0]

In [34]:
nonzero_values

[1, 2, 3]

Равносильный (прям равносильный) вариант сделать то же самое без `filter`:

In [35]:
nonzero_values = (v for v in values if v >= 0)

In [36]:
nonzero_values

<generator object <genexpr> at 0x0000029FB0E20678>

In [37]:
list(nonzero_values)

[1, 2, 3]

Дело в том, что выражение `(v for v ...)` — это ещё один вариант создания генератора... (`yield` есть, "где-то там")